Test for random walk with steps from a Gaussian distribution
================================

This notebook simulates a random walk with $3$ steps drawn from a mean zero normal distribution. Python's random package has methods for simulating a normal distribution. Since ordinal patterns are scale invariant, the standard deviation parameter won't affect the outcome.

We simulate a random walk 100,000 times and record the ordinal pattern for each iteration. The frequency of each ordinal pattern can then compared to calculations based on Theorem 4.1 of Bandt and Shiha's *Order patterns in time series* paper. For comparison, we look at both KL-divergence and the total variation distance. The primary purpose of the notebook is a sanity check - if we've made errors in our papers, the error should show up in an empirical test. 

In [1]:
%run ../src/gaussian_calculations.py
%run ../src/comparison.py
%run ../src/simulation.py
%run ../src/string_formatters.py

First we store what we believe is the true distribution for ordinal patterns in $S_4$ with steps drawn from a mean zero normal distribution. The calculation below uses Bandt and Shiha's generalization of this to arbitrary fractional Brownian motion given in *Order patterns in time series*.

In [2]:
Gaussian_distribution = gaussian4(0.5)

Other sources have given this calculation before. For example, DeFord and Moore's *Random walk null models for time series* and [this](https://mathoverflow.net/questions/97875/random-permutations-from-brownian-motion) question on Mathoverflow. Let's see what it looks like for the sake of comparison:

In [3]:
print(float_dictionary_string(Gaussian_distribution))

(1, 2, 3, 4) -> 0.1250     (3, 1, 4, 2) -> 0.0146     (4, 2, 3, 1) -> 0.0417 
(2, 1, 4, 3) -> 0.0270     (1, 2, 4, 3) -> 0.0625     (1, 4, 2, 3) -> 0.0208 
(3, 1, 2, 4) -> 0.0355     (1, 4, 3, 2) -> 0.0270     (1, 3, 2, 4) -> 0.0417 
(1, 3, 4, 2) -> 0.0355     (2, 1, 3, 4) -> 0.0625     (2, 3, 1, 4) -> 0.0208 
(2, 3, 4, 1) -> 0.0270     (2, 4, 1, 3) -> 0.0146     (2, 4, 3, 1) -> 0.0355 
(3, 2, 1, 4) -> 0.0270     (3, 2, 4, 1) -> 0.0208     (3, 4, 1, 2) -> 0.0270 
(3, 4, 2, 1) -> 0.0625     (4, 1, 2, 3) -> 0.0270     (4, 1, 3, 2) -> 0.0208 
(4, 2, 1, 3) -> 0.0355     (4, 3, 1, 2) -> 0.0625     (4, 3, 2, 1) -> 0.1250 



Now we run a random walk simulation using steps drawn from a mean zero normal distribution.

In [4]:
Gaussian_sim =  ordinal_gaussian_walk(4,100000) #This takes about 6 seconds on my older machine

Next we verify that the distance between the simulated distribution and the true distribution is small in both the total variation distance and a version of the KL-divergence recommended in [DeFord and Moore](https://arxiv.org/abs/1710.02175).

In [5]:
print(total_variation_distance(Gaussian_distribution, Gaussian_sim))
print(kullback_leibler(Gaussian_distribution, Gaussian_sim))

0.005333247356288836
7.8839675631e-05


Both values are small, as expected, though by itself, hard to interpret. For contrast, we'll compare to pure noise and then to a simulation of a random walk with Laplace and uniform steps.

In [6]:
from itertools import permutations
Noise_distribution = {p : 1/24 for p in permutations(range(1,5))} # Ordinal patterns are equidistributed in white noise.

In [7]:
print(total_variation_distance(Gaussian_distribution, Noise_distribution)) 
print(kullback_leibler(Gaussian_distribution, Noise_distribution))

0.2500000000000002
0.194605313847


In [8]:
Laplace_sim = ordinal_laplace_walk(4,100000)

In [9]:
print(total_variation_distance(Gaussian_distribution,Laplace_sim))
print(kullback_leibler(Gaussian_distribution,Laplace_sim))

0.02680649471257764
0.00420319320901


In [10]:
Uniform_sim = ordinal_uniform_walk(4,100000)

In [11]:
print(total_variation_distance(Gaussian_distribution, Uniform_sim))
print(kullback_leibler(Gaussian_distribution, Uniform_sim))

0.04010683862075574
0.00933966417722


Unsurprisingly, the distance to noise is greater than to the closely related random walks. Nonetheless, by these metrics, a simulation of a random walk with normal steps is much closer to the true distribution than a simulation from other distributions.